In [1]:
import pandas as pd
import numpy as np

In [2]:
b0,b1 = 77,84 # kohort til barn
f0, f1 = 40, 63 # kohort til foreldre 
alder = 31 # alder inntekt til barn blir målt
flytte_voksen = False # flytting før 30 hvis voksen, ellers før 18
m = 150 #antall kommuner

In [3]:
df_ssb = pd.read_csv('ssb_tabell.csv', header=None)
df_ssb = df_ssb.iloc[:-1,1:]

In [4]:
df = df_ssb.sort_values(by=2, ascending=False)
df['region'] = [x.split()[0] for x in df[1]]
df.index = range(len(df))
df_keep = df[:m]
df_drop = df[m:]
kommuner = list(pd.to_numeric(df_keep.region))

In [5]:
df = pd.read_csv('kpi.csv', header=0, sep=';')
kpi = df[df.columns[1]]
kpi.index = df[df.columns[0]]
kpi = kpi[kpi.index <= 2015]
kpi = kpi.str.replace(',','.')
kpi = kpi.astype(float)
kpi = 100 / kpi
kpi.head()

Unnamed: 0
2015    1.000000
2014    1.021450
2013    1.042753
2012    1.064963
2011    1.071811
Name: Årsgj.snitt, dtype: float64

##### nasjonal inntekt

In [6]:
with open('flytting.txt', 'w') as f:
    f.write('create-dataset flytting \n')
    f.write('import BEFOLKNING_FOEDSELS_AAR_MND as fodmnd \n')
    f.write('generate kohort = int( fodmnd / 100 ) \n')
    f.write(f'drop if (kohort > 19{b1}) | (kohort < 19{b0}) \n')
    f.write('import BEFOLKNING_INNALDER as innalder \n')
    f.write('keep if innalder == 0 \n')
    f.write('generate flyttetidspunkt = 0 \n')
    f.write('generate flytter  = 0 \n')

In [7]:
with open('flytting.txt', 'a') as f:
    for år in range(92,99+1):
        f.write(f'import BOSATTEFDT_BOSTED 19{år}-01-01 as kommune{år} \n')
        f.write(f'drop if sysmiss(kommune{år}) \n')
        f.write(f'destring kommune{år}, force \n')
    for år in range(0,4+1):
        f.write(f'import BOSATTEFDT_BOSTED 200{år}-01-01 as kommune0{år} \n')
        f.write(f'drop if sysmiss(kommune0{år}) \n')
        f.write(f'destring kommune0{år}, force \n')
    if flytte_voksen:
        for år in range(5,9+1):
            f.write(f'import BOSATTEFDT_BOSTED 200{år}-01-01 as kommune0{år} \n')
            f.write(f'drop if sysmiss(kommune0{år}) \n')
            f.write(f'destring kommune0{år}, force \n')
        for år in range(10,15+1):
            f.write(f'import BOSATTEFDT_BOSTED 20{år}-01-01 as kommune{år} \n')
            f.write(f'drop if sysmiss(kommune{år}) \n')
            f.write(f'destring kommune{år}, force \n')
    # fikse for kommunesammenslåinger
    for år in range(92,95+1):
        f.write(f"replace kommune{år} = 106 if (kommune{år} == 103) | (kommune{år} == 113) | (kommune{år} == 131) | (kommune{år} ==  133) | (kommune{år} == 134) \n")
    if flytte_voksen:
        for år in range(92,99):
            f.write(f"replace kommune{år} = 1804 if (kommune{år} == 1842) \n") 
            f.write(f"replace kommune{år} = 1505 if (kommune{år} == 1503) | (kommune{år} == 1556)\n") 
            f.write(f"replace kommune{år} = 1903 if (kommune{år} == 1901) | (kommune{år} == 1915)\n") 
        for år in range(5+1):
            f.write(f"replace kommune0{år} = 1804 if (kommune0{år} == 1842) \n") 
            f.write(f"replace kommune0{år} = 1505 if (kommune0{år} == 1503) | (kommune0{år} == 1556)\n") 
            f.write(f"replace kommune0{år} = 1903 if (kommune0{år} == 1901) | (kommune0{år} == 1915)\n") 
        for år in range(6,8+1):
            f.write(f"replace kommune0{år} = 1505 if (kommune0{år} == 1503) | (kommune0{år} == 1556)\n")
            f.write(f"replace kommune0{år} = 1903 if (kommune0{år} == 1901) | (kommune0{år} == 1915)\n") 
        for år in range(9,9+1):
            f.write(f"replace kommune0{år} = 1903 if (kommune0{år} == 1901) | (kommune0{år} == 1915)\n")
        for år in range(10,13+1):
            f.write(f"replace kommune{år} = 1903 if (kommune{år} == 1901) | (kommune{år} == 1915)\n") 
            

##### Inntekt barn

In [8]:
with open('flytting.txt', 'a') as f:
    for år in range(3,9+1):
        f.write('import BEFOLKNING_REGSTAT 200'+str(år)+'-01-01 as regstat0'+str(år)+'\n')
    for år in range(10,15+1):
        f.write('import BEFOLKNING_REGSTAT 20'+str(år)+'-01-01 as regstat'+str(år)+'\n')

    for år in range(2003, 2015+1):
        f.write('import INNTEKT_WSAMINNT_KORRIGERT '+str(år)+'-01-01 as i'+str(år)[2:]+'\n')
        f.write('replace i'+str(år)[2:] +' = 0 if sysmiss(i'+str(år)[2:]+') \n')
        f.write('generate i'+str(år)[2:]+'ij = i'+str(år)[2:]+'*'+f'{kpi[år]:.4f}\n')
        
    for kohort in range(b0, b1+1):
        f.write(f"drop if (regstat{str(kohort+alder)[1:]} != '1') & kohort == {kohort} \n")
    
    f.write(f'generate innt{str(alder)} = 0 \n')
    for kohort in range(b0, b1+1):
        f.write(f'replace innt{str(alder)} = i'+str(kohort+alder)[1:]+'ij if kohort == 19'+str(kohort)+'\n')
    
    
    f.write(f'generate inntekt_barn = innt{str(alder)} \n')
    f.write('import BEFOLKNING_FAR_FNR as far_fnr \n')

##### Inntekt far

In [9]:
with open('flytting.txt', 'a') as f:
    f.write('create-dataset far \n')
    f.write('import BEFOLKNING_FOEDSELS_AAR_MND as fodmnd \n')
    f.write('generate kohortf = int( fodmnd / 100 ) \n')
    f.write(f'drop if kohortf > 19{f1} | kohortf < 19{f0} \n')
    f.write('import BEFOLKNING_KJOENN as mann \n')
    f.write("drop if mann == '2' \n")
    
    for år in range(93,98+1):
        f.write('import BEFOLKNING_REGSTAT 19'+str(år)+'-01-01 as regstat'+str(år)+'\n')
    
    string = "drop if "
    for år in range(93,97+1):
        string += f"(regstat{str(år)}!= '1') | "
    string += "(regstat98 != '1') \n"
    f.write(string)
        
    for år in range(1993, 1998+1):
        f.write('import INNTEKT_WSAMINNT_KORRIGERT '+str(år)+'-01-01 as if'+str(år)[2:]+'\n')
        f.write('replace if'+str(år)[2:] +' = 0 if sysmiss(if'+str(år)[2:]+') \n')
        f.write('generate if'+str(år)[2:]+'ij = if'+str(år)[2:]+'*'+f'{kpi[år]:.4f}\n')
    f.write('generate inntekt_far = (if93ij + if94ij + if95ij + if96ij + if97ij + if98ij) / 6 \n')
    f.write('replace inntekt_far = 0 if sysmiss(inntekt_far) \n')
    f.write('drop if sysmiss(inntekt_far) | inntekt_far <= 0 \n')
    f.write('generate prosentil_far = quantile(inntekt_far, 100) \n')
    f.write('merge prosentil_far into flytting on far_fnr \n')
    f.write('use flytting \n')
    f.write('drop if sysmiss(prosentil_far) \n')
    f.write('generate prosentil_barn = quantile(inntekt_barn, 100) \n')

##### Flytting

In [10]:
with open('flytting.txt', 'a') as f:
    string = "keep if"
    for i in range(len(kommuner)-1):
        string += f" (kommune92 == {kommuner[i]}) |"
    string += f" (kommune92 == {kommuner[-1]}) \n"
    f.write(string)   

In [11]:
if not flytte_voksen:
    with open('flytting.txt', 'a') as f:
        f.write('generate destinasjon = 0 \n')

    with open('flytting.txt', 'a') as f:
        for år in range(93,99+1):
            f.write(f'replace flytter = flytter + 1 if (kommune{år} != kommune{år-1}) & (19{år} - kohort <= 18) \n')
            f.write(f'replace destinasjon = kommune{år} if (flytter == 1) \n') 
            f.write(f'replace flyttetidspunkt = {1900+år-1} if (kommune{år} != kommune{år-1})  & (flytter == 1) & (19{år} - kohort <= 18) \n')      

    # håndtere 00
    with open('flytting.txt', 'a') as f:
        f.write('replace flytter = flytter + 1 if (kommune00 != kommune99) & (2000 - kohort <= 18) \n')
        f.write(f'replace destinasjon = kommune00 if (flytter == 1)  \n') 
        f.write('replace flyttetidspunkt = 1999 if (kommune00 != kommune99) & (flytter == 1) & (2000 - kohort <= 18) \n')
    with open('flytting.txt', 'a') as f:
        for år in range(1,4+1):
            f.write(f'replace flytter = flytter + 1 if (kommune0{år} != kommune0{år-1}) & (200{år} - kohort <= 18) \n')
            f.write(f'replace destinasjon = kommune0{år} if (flytter == 1) \n') 
            f.write(f'replace flyttetidspunkt = {2000+år-1} if (kommune0{år} != kommune0{år-1}) & (flytter == 1) & (200{år} - kohort <= 18) \n')
        f.write('tabulate kommune04 \n')

In [12]:
if flytte_voksen:
    with open('flytting.txt', 'a') as f:
        f.write('generate destinasjon = 0 \n')

    with open('flytting.txt', 'a') as f:
        for år in range(93,99+1):
            f.write(f'replace flytter = flytter + 1 if (kommune{år} != kommune{år-1}) & (19{år} - kohort <= 30) \n')
            f.write(f'replace destinasjon = kommune{år} if (flytter == 1) & (19{år} - kohort == 30) \n') 
            f.write(f'replace flyttetidspunkt = {1900+år-1} if (kommune{år} != kommune{år-1})  & (flytter == 1) & (19{år} - kohort <= 30) \n')      

    # håndtere 00
    with open('flytting.txt', 'a') as f:
        f.write('replace flytter = flytter + 1 if (kommune00 != kommune99) & (2000 - kohort <= 30) \n')
        f.write(f'replace destinasjon = kommune00 if (flytter == 1) & (2000 - kohort == 30) \n') 
        f.write('replace flyttetidspunkt = 1999 if (kommune00 != kommune99) & (flytter == 1) & (2000 - kohort <= 30) \n')
    with open('flytting.txt', 'a') as f:
        for år in range(1,9+1):
            f.write(f'replace flytter = flytter + 1 if (kommune0{år} != kommune0{år-1}) & (200{år} - kohort <= 30) \n')
            f.write(f'replace destinasjon = kommune0{år} if (flytter == 1) & (200{år} - kohort == 30) \n') 
            f.write(f'replace flyttetidspunkt = {2000+år-1} if (kommune0{år} != kommune0{år-1}) & (flytter == 1) & (200{år} - kohort <= 30) \n')
    # håndtere 10
    with open('flytting.txt', 'a') as f:
        f.write('replace flytter = flytter + 1 if (kommune10 != kommune09) & (2010 - kohort <= 30) \n')
        f.write(f'replace destinasjon = kommune10 if (flytter == 1) & (2010 - kohort == 30) \n') 
        f.write('replace flyttetidspunkt = 2009 if (kommune10 != kommune09) & (flytter == 1) & (2010 - kohort <= 30) \n')
    with open('flytting.txt', 'a') as f:
        for år in range(11,15+1):
            f.write(f'replace flytter = flytter + 1 if (kommune{år} != kommune{år-1}) & (20{år} - kohort <= 30) \n')
            f.write(f'replace destinasjon = kommune{år} if (flytter == 1) & (20{år} - kohort == 30) \n') 
            f.write(f'replace flyttetidspunkt = {2000+år-1} if (kommune{år} != kommune{år-1}) & (flytter == 1) & (20{år} - kohort <= 30) \n')
        f.write('tabulate kommune04 \n')

In [13]:
# IKKE KJØR PÅ FØRSTE
with open('flytting.txt', 'a') as f:
    string = "keep if"
    for i in range(len(kommuner)-1):
        string += f" (destinasjon == {kommuner[i]}) |"
    string += f" (destinasjon == {kommuner[-1]}) \n"
    f.write(string)

In [14]:
with open('flytting.txt', 'a') as f:
    f.write('generate flyttealder = flyttetidspunkt-kohort if flytter == 1 \n')
    f.write('tabulate flytter \n')
    f.write('tabulate flyttealder \n')

##### Lage delta

In [15]:
x = 'v' if flytte_voksen else 'b'
df = pd.read_csv(f'df_{x}_{alder}.csv',index_col=0)

In [17]:
koef_dict = {}
for key in kommuner:
    try:
        koef_dict[key] = [x for x in df.loc[key]]
    except:
        pass

In [18]:
with open('flytting.txt', 'a') as f:
    f.write('keep if flytter == 1 \n')

    f.write('generate y_po = 99 \n')
    f.write('generate y_pd = 99 \n')
    
    for key in koef_dict:
        f.write(f'replace y_po = {koef_dict[key][0]:.4f} + {koef_dict[key][1]:.4f} * prosentil_far if kommune92 == {key} \n')
        f.write(f'replace y_pd = {koef_dict[key][0]:.4f} + {koef_dict[key][1]:.4f} * prosentil_far if destinasjon == {key} \n')
    f.write('drop if y_po == 99 \n')
    f.write('drop if y_pd == 99 \n')
    f.write('generate delta = y_pd-y_po \n')

In [19]:
with open('flytting.txt', 'a') as f:
    f.write('tabulate flyttealder \n')
    f.write('tabulate kommune92 \n')
    f.write('tabulate destinasjon \n')
    
    f.write('histogram delta \n')

    f.write('tabulate kommune92, summarize(delta) \n')
    f.write('tabulate destinasjon, summarize(delta) \n')
    
    f.write('regress prosentil_barn y_po i.flyttealder#c.delta c.prosentil_far#i.flyttealder i.flyttealder \n')
    f.write('regress prosentil_barn y_po i.flyttealder#c.delta c.prosentil_far#i.flyttealder delta i.flyttealder \n')
    
    f.write('generate diff18 = 18-flyttealder \n')
    
    f.write('regress prosentil_barn y_po c.diff18#c.delta c.prosentil_far#i.flyttealder i.flyttealder \n')
    f.write('regress prosentil_barn y_po c.diff18#c.delta delta c.prosentil_far#i.flyttealder i.flyttealder \n')
    f.write('regress prosentil_barn y_po c.diff18#c.delta delta c.prosentil_far#i.flyttealder i.flyttealder, noconstant \n') 

In [20]:
with open('flytting.txt', 'a') as f:
    f.write('generate diff_delta = diff18*delta \n')
    
    f.write('clone-dataset flytting flytting1 \n')
    f.write('use flytting1 \n')
    f.write('collapse (mean) diff_delta, by(far_fnr) \n')
    f.write('rename diff_delta diff_delta_snitt \n')
    f.write('merge diff_delta_snitt into flytting on far_fnr \n')
    f.write('use flytting \n')
    f.write('generate diff_delta_avvik = diff_delta - diff_delta_snitt \n')
    
    f.write('clone-dataset flytting flytting2 \n')
    f.write('use flytting2 \n')
    f.write('collapse (mean) prosentil_barn, by(far_fnr) \n')
    f.write('rename prosentil_barn prosentil_snitt \n')
    f.write('merge prosentil_snitt into flytting on far_fnr \n')
    f.write('use flytting \n')
    f.write('generate prosentil_avvik = prosentil_barn - prosentil_snitt \n')
    
    f.write('clone-dataset flytting flytting3 \n')
    f.write('use flytting3 \n')
    f.write('collapse (mean) flyttealder, by(far_fnr) \n')
    f.write('rename flyttealder flyttealder_snitt \n')
    f.write('merge flyttealder_snitt into flytting on far_fnr \n')
    f.write('use flytting \n')
    f.write('generate flyttealder_avvik = flyttealder-flyttealder_snitt \n')
    f.write('regress prosentil_avvik diff_delta_avvik if diff_delta_avvik != 0 & prosentil_avvik != 0 \n')
    f.write('regress prosentil_avvik diff_delta_avvik flyttealder_avvik if diff_delta_avvik != 0 & prosentil_avvik != 0 \n')
    f.write('regress prosentil_avvik diff_delta_avvik if diff_delta_avvik != 0 & prosentil_avvik != 0, noconstant \n')